# check oof df

In [17]:
import os

import numpy as np
import pandas as pd
from pandarallel import pandarallel
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [19]:
output_dir = os.path.join("/kaggle", "working", "_oof")
# exp_name = "baseline000"
# exp_name = "baseline000_1697037446.0796616"
# exp_name = "baseline_alldata_allfold_000"
exp_name = "exp000"
# oof_df_path = os.path.join(output_dir, exp_name,"oof_df.parquet")
# oof_df = pd.read_parquet(oof_df_path)
for i in range(5):
    print("fold", i)
    oof_df = pd.concat(
        [pd.read_parquet(os.path.join(output_dir, exp_name, f"oof_df_fold{i}.parquet"))]
        ,axis=0
    )

fold 0
fold 1
fold 2
fold 3
fold 4


In [20]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

# 1step 0.5secで30minなら60*30=1800step
def postprocess_fn(df, N=1800, maxpool_kernel_size=101, maxpool_stride=1):
    df = df.copy()
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x.rolling(N, min_periods=1).mean())
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1])
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - maxpool_stride) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(maxpool_kernel_size, stride=maxpool_stride, padding=maxpool_padding)
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)
    return df


def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [21]:
# df = postprocess_fn(oof_df, N=1200)
# sub_df = make_submission_df(df, threshold=0.2)

In [22]:
# data_num = 20000
# for _ in range(5):
#     random_num = np.random.randint(0, len(df)-data_num)
#     print(random_num)
#     plt.figure(figsize=(20, 5))
#     plt.subplot(3, 1, 1)
#     plt.plot(df["step"].iloc[random_num:random_num+data_num], df["event_pred"].iloc[random_num:random_num+data_num], label="class_pred")
#     plt.subplot(3, 1, 2)
#     plt.plot(df["step"].iloc[random_num:random_num+data_num], df["event"].iloc[random_num:random_num+data_num], label="event_pred")
#     plt.subplot(3, 1, 3)
#     plt.plot(df["step"].iloc[random_num:random_num+data_num], df["class_pred"].iloc[random_num:random_num+data_num], label="class_pred")


In [23]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [24]:
import sys
sys.path.append("/kaggle/src/dss_utils")

# series_id_column_name = "series_id"
# time_column_name = "step"
# event_column_name = "event"
# score_column_name = "score"
# use_scoring_intervals = None  # type:ignore


# # [1, 3, 5, 7, 10, 13, 15, 20, 25, 30] minute
# tolerances = {
#     "onset": [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],
#     "wakeup": [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],
# }

from dss_metrics import score

In [25]:
event_df = train_event_df[train_event_df["step"].notnull()].copy()

In [26]:
average_size = 200
max_kernel_size = 101
df = postprocess_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
sub_df = make_submission_df(df, threshold=0.1)

In [27]:
score(event_df, sub_df)

0.6494387768263943

In [28]:
# for average_size in range(101, 151, 201):
#     for max_kernel_size in range(11, 101, 10):
#         print("average_size", average_size, "max_kernel_size", max_kernel_size)
#         df = postprocess_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
#         sub_df = make_submission_df(df, threshold=0.1)
#         print(score(event_df, sub_df))

In [29]:
# for average_size in [201, 151]:
#     for max_kernel_size in range(11, 101, 10):
#         print("average_size", average_size, "max_kernel_size", max_kernel_size)
#         df = postprocess_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
#         sub_df = make_submission_df(df, threshold=0.1)
#         print(score(event_df, sub_df))

In [30]:
# max_kernel_size = 41
# for average_size in range(301, 601, 100):
#     print("average_size", average_size, "max_kernel_size", max_kernel_size)
#     df = postprocess_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
#     sub_df = make_submission_df(df, threshold=0.1)
#     print(score(event_df, sub_df))

In [31]:
max_kernel_size = 41
for average_size in range(201, 401, 20):
    print("average_size", average_size, "max_kernel_size", max_kernel_size)
    df = postprocess_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    print(score(event_df, sub_df))

average_size 201 max_kernel_size 41
0.6507715513863849
average_size 221 max_kernel_size 41
0.6537339796886537
average_size 241 max_kernel_size 41
0.6561407703011912
average_size 261 max_kernel_size 41
0.6576799761907189
average_size 281 max_kernel_size 41
0.6575245455595717
average_size 301 max_kernel_size 41
0.6577504639656441
average_size 321 max_kernel_size 41
0.6569745968622127
average_size 341 max_kernel_size 41
0.6562378869455485
average_size 361 max_kernel_size 41
0.6552772069687894
average_size 381 max_kernel_size 41
0.6557543937388506


In [32]:
max_kernel_size = 41
for average_size in range(401, 1001, 100):
    print("average_size", average_size, "max_kernel_size", max_kernel_size)
    df = postprocess_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    print(score(event_df, sub_df))

average_size 401 max_kernel_size 41
0.6550113465426012
average_size 501 max_kernel_size 41
0.6551176525789024
average_size 601 max_kernel_size 41
0.6541576011134318
average_size 701 max_kernel_size 41
0.6542691205014803
average_size 801 max_kernel_size 41
0.6582780850952523
average_size 901 max_kernel_size 41
0.6561059229388471


## exp 000
average_size 201 max_kernel_size 41
0.6507715513863849
average_size 221 max_kernel_size 41
0.6537339796886537
average_size 241 max_kernel_size 41
0.6561407703011912
average_size 261 max_kernel_size 41
0.6576799761907189
average_size 281 max_kernel_size 41
0.6575245455595717
average_size 301 max_kernel_size 41
0.6577504639656441
average_size 321 max_kernel_size 41
0.6569745968622127
average_size 341 max_kernel_size 41
0.6562378869455485
average_size 361 max_kernel_size 41
0.6552772069687894
average_size 381 max_kernel_size 41
0.6557543937388506

# exp001
average_size 401 max_kernel_size 41
0.6312146268156249
average_size 501 max_kernel_size 41
0.635633422232492
average_size 601 max_kernel_size 41
0.6368531157055373
average_size 701 max_kernel_size 41
0.6369187233473134
average_size 801 max_kernel_size 41
0.6415632079075928
average_size 901 max_kernel_size 41
0.6409006872463292

# exp002
average_size 201 max_kernel_size 41
0.6311561301629709
average_size 221 max_kernel_size 41
0.6338846022029172
average_size 241 max_kernel_size 41
0.6339425602342341
average_size 261 max_kernel_size 41
0.6342376545266758
average_size 281 max_kernel_size 41
0.6348944831498222
average_size 301 max_kernel_size 41
0.6343623084738816
average_size 321 max_kernel_size 41
0.6340311850884977
average_size 341 max_kernel_size 41
0.6329142802968093
average_size 361 max_kernel_size 41
0.632642525476107
average_size 381 max_kernel_size 41
0.6316554793826874